In [42]:
from common import *
import warnings 
warnings.filterwarnings('ignore')
from pathlib import Path
import json, copy

In [43]:
I = lambda x: x
logavg = lambda x,y: math.expm1((math.log1p(x)+math.log1p(y))/2)
keepSigFig = lambda n: lambda x: round(x, -int(math.floor(math.log10(abs(x)))) + (n - 1)) if x else x

# #!#!#!#!#!#!#!#!#!#!#! Save System #!#!#!#!#!#!#!#!#!#!#!

### File System

In [223]:
class IndexFileSystem():
    def __init__(self, directory, key):
        self.dir = directory if isinstance(directory, Path) else Path(directory)
        self.dir.mkdir(exist_ok=True, parents=True)
        assert isinstance(key, str), "key must be string"
        self.key = key
        self.io = O()

    def iterIndices(self):
        for dot_params in self.dir.glob('*.' + self.key):
            yield int(dot_params.stem)

    def getFilePath(self, name, *, i):
        return self.dir / (str(i) + '.' + name)

    @staticmethod
    def readWrapper(read):
        '''wraps io read operations to safely return None if file does not exist'''
        @wraps(read)
        def read_safely(*a, **k):
            try:
                return read(*a, **k)
            except FileNotFoundError:
                return None
        return read_safely

    def assignIO(self, name, *, read, write, format='custom'):
        assert format in ['bytes', 'text', 'custom'], "argument `format` must be one of 'bytes' or 'text' or 'custom'"
        if format in ['bytes', 'text']:
            self.io[name] = O(
                read = self.readWrapper( lambda *,i: read(getattr(self.getFilePath(name, i=i), 'read_'+format)()) ),
                write = lambda x,*,i: getattr(self.getFilePath(name, i=i), 'write_'+format)(write(x))
            )
        elif format == 'custom':
            def readCustom(*, i):
                file = self.getFilePath(name, i=i)
                if not file.exists():
                    return None
                try:
                    return read(file)
                except Exception:
                    return read(str(file))
            def writeCustom(*, i):
                file = self.getFilePath(name, i=i)
                try:
                    write(x, file=file)
                except Exception:
                    write(x, file=str(file))
            self.io[name] = O(read=readCustom, write=writeCustom)

class IndexDataStore():    
    def __init__(self, file, _factory_=False):
        if not _factory_:
            assert False, "Cannot instantiate IndexDataStore normally. Please use factory static method."
        self.file = file
        self.op = O()
        self.lists = O()
        self.key = None
        self.keyFunc = lambda keyVal, *, client=False: object()
        self.tbl = {}
        self.nextIndex = 0
        
    def load(self):
        indices = sorted(self.file.iterIndices())
        n = self.nextIndex = (max(indices) if indices else -1) + 1
        for name, _ in dict.items(self.lists):
            self.lists[name] = [None] * self.nextIndex
        
        for i in indices:
            for name in self.lists:
                if self.op[name].load:
                    self.op[name].load(i=i)
                
        if self.key is not None:
            self.assignKey(self.key, self.keyFunc)
            
    def i(self, key, dry=False, client=True, keep=True, save=True):
        keyVal = self.keyFunc(key, client=client)
        
        if keyVal not in self.tbl and not dry:
            for name in self.lists:
                assert len(self.lists[name]) == self.nextIndex, "idk internal error"
                self.lists[name].append(None)
            self.tbl[keyVal] = self.nextIndex
            if save and self.op[self.key].save:
                self.op[self.key].save(key, i=self.nextIndex, keep=keep)
            elif keep and self.op[self.key].keep:
                self.op[self.key].keep(key, i=self.nextIndex, client=client)
            self.nextIndex += 1
        elif keyVal not in self.tbl and dry:
            return None
        
        return self.tbl[keyVal]
            
    def save(self, keep=True, **kwargs):
        '''`client` is always True'''
        assert ('i' in kwargs) ^ (self.key in kwargs), "call to `save` must include exactly one of i= or the key name ="
        if 'i' in kwargs:
            i = kwargs['i']
            del kwargs['i']
        else:
            i = self.i(kwargs[self.key])
        
        for name in kwargs:
            if name == self.key: # already saved in `self.i(save=True)` above
                continue
            if name not in self.op:
                raise AssertionError(f"given save item '{name}' does not have data store ops initialized")
            if self.op[name].save:
                self.op[name].save(kwargs[name], i=i, keep=keep)
            
    def get(self, name, **kwargs):
        assert len(kwargs) == 1 and ('i' in kwargs or self.key in kwargs), "must give assigned 'key' or i"
        if 'i' in kwargs:
            i = kwargs['i']
            del kwargs['i']
        else:
            i = self.i(kwargs[self.key], dry=True)
        if i is None:
            return None
        return self.lists[name][i]
        
    ##################### INSTANCE BUILDING METHODS ##################### : 
    def assignKey(self, name, func):
        assert name in self.lists, "key name must first be assigned ops"
        self.key = name
        self.keyFunc = func
        self.tbl = {self.keyFunc(x): i for i,x in enumerate(self.lists[self.key]) if x is not None}
        
    def assignOperations(self, name, *, load=I, save=I, keep=I, keepSave=None, keepClient=None):
        if keepClient is None:
            keepClient = keep
            
        #NOTE #TODE? the method names below conflict with the local vars up here, and current Python syntax takes
        # the variables up here. Should this change the code below will stop working. But this is the "nicest" way to do it
        class the(O()):
            def load(*, i, keep=True):
                if not load:
                    return None
                x = load(self.file.io[name].read(i=i))
                if keep and self.op[name].keep:
                    self.lists[name][i] = self.op[name].keep(x, i=i)
                return x
            
            def save(x, *, i, keep=True):
                if not save:
                    return
                saved_x = save(x)
                self.file.io[name].write(saved_x, i=i)
                if keep and self.op[name].keep:
                    try:
                        self.lists[name][i] = self.op[name].keep(saved_x, i=i, save=True)
                        assert self.lists[name][i] is not None
                    except (TypeError, AssertionError):
                        self.lists[name][i] = self.op[name].keep(x, i=i, client=True)
                
            def keep(x, *, i, save=False, client=False):
                assert not (save and client), "only one of `save` and `client` can be specified"
                if not any([keep, keepSave, keepClient]):
                    return None
                kept = None
                if save and keepSave:
                    kept = keepSave(x)
                elif client and keepClient:
                    kept = keepClient(x)
                elif keep:
                    kept = keep(x)
                if kept is not None:
                    self.lists[name][i] = kept
                return kept
            
        if not load:
            the.load = False
        if not save:
            the.save = False
        if not any([keep, keepSave, keepClient]):
            the.keep = False
            
        self.op[name] = the
        if name not in self.lists:
            self.lists[name] = [None] * self.nextIndex
    
    
@staticmethod
def __IndexDataStore__from_specs(specs, **kwargs):
    for k, v in kwargs.items():
        specs[k] = v
    
    f = IndexFileSystem(specs.dir, specs.key)
    d = IndexDataStore(f, _factory_=True)
    
    _readwriteformat = {'read', 'write', 'format'}
    _loadsavekeep = {'load', 'save', 'keep', 'keepSave', 'keepClient'}
    for name, val in dict.items(specs.op):
        f.assignIO(name, **{a: b for a,b in dict.items(val) if a in _readwriteformat})
        d.assignOperations(name, **{a: b for a,b in dict.items(val) if a in _loadsavekeep})
        
    keyFuncClient = specs.keyFuncClient if 'keyFuncClient' in specs else specs.keyFunc
    keyFunc = lambda keyVal, *, client=False: keyFuncClient(keyVal) if client else specs.keyFunc(keyVal)
    d.assignKey(specs.key, keyFunc)
    
    d.load()
    return d
IndexDataStore.from_specs = __IndexDataStore__from_specs


def json_default(o):
    if isinstance(o, np.int64):
        return int(o)  
    raise TypeError
    
class __IndexDataStore__SpecsHelper(metaclass=staticclass):
    def json(op):
        op.format = 'text'
        op.read = json.loads
        op.write = lambda x: json.dumps(x, default=json_default)
        return op
    def pickle(op):
        op.format = 'bytes'
        op.read = pickle.loads
        op.write = pickle.dumps
        return op
IndexDataStore.SpecsHelper = __IndexDataStore__SpecsHelper
IDSSH = IndexDataStore.SpecsHelper

### Model Manager

In [224]:
class ModelManager():
    features_dir = '.features'
    samples_dir = '.samples'
    models_dir = 'models'
    
    def __init__(self, directory, f_specs=None, s_specs=None, p_specs=None): #GLOBAL FDS SDS PDS
        global FDS, SDS, PDS
        if f_specs is None:
            f_specs = FDS
        if s_specs is None:
            s_specs = SDS
        if p_specs is None:
            p_specs = PDS
        self.p_specs = p_specs
        self.dir = directory if isinstance(directory, Path) else Path(directory)
        self.dir.mkdir(exist_ok=True) # no parents because this is user facing and enabling parents could get gnarly
        self.F = IndexDataStore.from_specs(f_specs, dir=self.dir/self.features_dir)
        self.S = IndexDataStore.from_specs(s_specs, dir=self.dir/self.samples_dir)
        (self.dir/self.models_dir).mkdir(exist_ok=True)
        self.load()
        
    def load(self):
        self.pms = {}
        for fi, si in self.iterIndices():
            self.make_params_manager(i=(fi, si))
        
    def iterIndices(self):
        for fdir in (self.dir/self.models_dir).iterdir():
            try:
                fi = int(fdir.name)
            except ValueError:
                continue
            assert fdir.is_dir(), "folder in models folder whose name is just a number must be a folder"
            assert fi < self.F.nextIndex, "features folder found with greater index than what has been labelled"
            for sdir in fdir.iterdir():
                try:
                    si = int(sdir.name)
                except ValueError:
                    continue
                assert sdir.is_dir(), "folder in models folder 1 layer down whose name is just a number must be a folder"
                assert si < self.S.nextIndex, "samples folder found with greater index than what has been labelled"
                yield (fi, si)
        
    def make_params_manager(self, features=None, samples=None, i=None):
        assert (
            not ((features is None) ^ (samples is None)) and
            ((features is not None and samples is not None) ^ (i is not None)),
            "invalid arguments to `make_param_manager`"
        )
        if i is None:
            i = self.F.i(features), self.S.i(samples)
        fi, si = i
        assert 0 <= fi < self.F.nextIndex and 0 <= si < self.S.nextIndex, "bad arguments to `make_params_manager`"
        pm = IndexDataStore.from_specs(self.p_specs, dir=self.dir/self.models_dir/str(fi)/str(si))
        self.pms[fi, si] = pm
        return pm
    
    def i(features, samples): # purely convenience public interface
        return self.F.i(features), self.S.i(samples)
        
    def PM(self, features, samples, get_i=False):
        fi, si = self.F.i(features), self.S.i(samples)
        if get_i:
            return self.iPM(fi, si), (fi, si)
        else:
            return self.iPM(fi, si)
        
    def iPM(self, fi, si):
        if (fi, si) not in self.pms:
            self.make_params_manager(i=(fi, si))
        return self.pms[fi, si]

### SPECS for managers

In [225]:
lgbNullDataset = lgb.Dataset(pd.DataFrame({'_a_': np.arange(100), '_b_': np.arange(100)}))

In [310]:
class PDS(O()):
    '''data transformation code inside `op` object:

    client --save(+write)--> disk; client --keep--> memory; disk --load(+read)--> memory

    read = text/bytes stream -> as-is object read from file
    write = object to save as is to file -> text/bytes stream
    load = as-is object read from file -> object to be loaded in memory
    save = raw object given by client -> object to save as-is to file
    keep = as-is object read from file -> object to keep in memory
    keepClient = raw object given by client -> object to keep in memory
    '''

    key = 'Params'
    keyFunc = lambda params: tuple(sorted(dict.items(params)))

    class op(O()):
        @IDSSH.json
        class Params(O()):
            '''just a dict of the parameter value assignments'''
            keepClient = dict
            
        @IDSSH.pickle
        class Results(O()):
            '''should be a dict-like of various things, most importantly including "score"'''
            keepClient = copy.deepcopy
            
        @IDSSH.pickle
        class Training(O()):
            '''tuple (aligning with samples training/cv split tuple) of LightGBM training eval DataFrames'''
            keep = False
            
        @IDSSH.pickle
        class Boosters(O()):
            '''the actual lgb.Booster model. well, a tuple of them, one for each cv set'''
            #read = lambda file: lgb.Booster(model_file=file)
            load = lambda x: tuple(lgb.Booster(train_set=lgbNullDataset).model_from_string(s, verbose=False) for s in x)
            #write = lambda x, file: x.save_model(file)
            save = lambda x: tuple(b.model_to_string() for b in x)
            keep = False

In [311]:
frzset = lambda x: x if isinstance(x, frozenset) else frozenset(x)
pydict = lambda x: copy.deepcopy(dict(**x) if type(x) is not dict else x)

class FDS(O()):
    key = 'Feats'
    keyFunc = lambda features: features # internally already coverted to immutable
    keyFuncClient = frzset
    class op(O()):
        @IDSSH.json
        class Feats(O()):
            load = frozenset
            save = sorted
            keepClient = frozenset
    
class SDS(O()):
    key = 'Samps'
    keyFunc = lambda samples: samples # internally already converted immutable
    keyFuncClient = lambda samples: tuple((frzset(tr), frzset(cv)) for tr, cv in samples)
    class op(O()):
        @IDSSH.json
        class Samps(O()):
            load = lambda samples: tuple((frzset(tr), frzset(cv)) for tr, cv in samples)
            save = lambda samples: [[sorted(tr), sorted(cv)] for tr, cv in samples]
            keepClient = load
        @IDSSH.json
        class Ctor(O()):
            save = pydict
            keepSave = copy.deepcopy
            keepClient = lambda code: copy.deepcopy(pydict(code))

# ~!~!~!~!~!~! Model (features/samples/parameters) Searching ~!~!~!~!~!~!~!

### parameter search logic

In [312]:
class ParamSearch():
    mix = staticmethod(logavg)
    
    def __init__(self, specs):
        self.specs = O.mycopy(specs)
        self.setup_specs()
        
    def setup_specs(self):
        dsc = self.specs.Discrete
        dsc.keys = list(flatten( dsc.enc.keys() ))
        dsc.assigns = [list(flatten(x)) for x in product(* dsc.enc.values() )]
        obo = self.specs.OneByOne
        for k,v in dict.items(obo.data):
            for i,x in dict.items(obo.default):
                if i not in v:
                    v[i] = x
    
    def search(self):
        dsc, obo = self.specs.Discrete, self.specs.OneByOne
        for assign in dsc.assigns:
            params = dict(zip(dsc.keys, assign))
            coroutine = self.one_by_one()
            for addon in coroutine:
                params.update(addon)
                coroutine.send((yield copy.deepcopy(params))); assert (yield) is None
        
        
    def one_by_one(self):
        obod = self.specs.OneByOne.data
        
        #! main algorithm ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        
        # initialize loop variables
        params = {k: (v.b if 'b' in v else v.a[1]) for k,v in dict.items(obod)}
        ranges = {k: v.a for k,v in dict.items(obod)}
        scores = {k: [-np.inf, -np.inf] for k in params}
        isdone = {k: False for k in params}
        
        # pre loop one-off work
        base_score = yield params; assert (yield) is None
        
        # loop
        for i in range(999_999_999):
            #! try new parameter values ############################## part A of loop work
            
            # initialize local loop variable
            new_scores = {k: [-np.inf, -np.inf] for k in params}
            new_params = {k: ( obod[k].cast(self.mix(v[0], v[1])),
                               obod[k].cast(self.mix(v[1], v[2])) ) for k,v in dict.items(ranges)}
            
            # finish condition check
            isdone = {k: v or i>=obod[k].lim for k,v in dict.items(isdone)}
            if all(isdone.values()):
                break
                
            # try new parameter values for all parameters
            for key in list(params):
                if i >= obod[key].lim:
                    continue
                orig = params[key]
                params[key] = new_params[key][0]
                scores[key][0] = yield params; assert (yield) is None
                params[key] = new_params[key][1]
                scores[key][1] = yield params; assert (yield) is None
                params[key] = orig
            
            #! start setting up values for next loop ######################## part B of loop work
            
            # set params to the best found and see if it betters score, updating ranges also
            #CODE num_nochange = 0
            for key in list(params):
                if scores[key][0] > base_score and scores[key][0] >= scores[key][1]:
                    params[key] = new_params[key][0]
                    ranges[key] = [ranges[key][0], params[key], ranges[key][1]]
                elif scores[key][1] > base_score and scores[key][1] >= scores[key][0]:
                    params[key] = new_params[key][1]
                    ranges[key] = [ranges[key][1], params[key], ranges[key][2]]
                else:
                    ranges[key] = [new_params[key][0], ranges[key][1], new_params[key][1]]
                    #num_nochange += 1
                
            # send out new params
            #CODE if num_nochange < len(params):
            base_score = yield params; assert (yield) is None

### Features / Samples(train/cv split) search

In [313]:
def KaggleMetric():
    def __init__(self, incr=0):
        self.incr = incr
    
    def attach(self, ms):
        L, s = ms._L, ms._s
        for Ltr, Lcv, tr, cv in zip(L.tr, L.cv, s.tr, s.cv):
            Ltr.timeFactor = ms.Y.time[tr].factorize()[0]
            Lcv.timeFactor = ms.Y.time[cv].factorize()[0]
            Ltr.value = (ms.Y.upDown1*ms.Y.absVal1)[tr]
            Lcv.value = (ms.Y.upDown1*ms.Y.absVal1)[cv]
            Ltr.i = 0
            Lcv.i = 0
    
    def __call__(self, preds, valid_data):
        df_time = valid_data.timeFactor
        #labels = valid_data.get_label()
        values = valid_data.value
        #assert len(labels) == len(df_time)

        preds = preds*2-1
        #labels = labels*2-1
        x_t = preds * values

        # Here we take advantage of the fact that `labels` (used to calculate `x_t`)
        # is a pd.Series and call `group_by`
        x_t_sum = x_t.groupby(df_time).sum()
        score = x_t_sum.mean() / x_t_sum.std()

        valid_data.i += self.incr
        return 'kaggle', score+valid_data.i, True

In [324]:
from sklearn.model_selection import GroupShuffleSplit, GroupKFold
        
class ModelSearch():
    
    def __init__(self, specs, mm, *, X, Y):
        self.specs = specs
        self.mm = mm
        self.X = X
        self.Y = Y
        self.setup_specs()
        
    def setup_specs(self):
        self.specs.Samples.data = []
        for code in self.specs.Samples.enc:
            if code.method == 'GroupShuffleSplit.2':
                tr, cv = next(GroupShuffleSplit(**code.kwargs).split(self.X, self.Y, groups=self.Y[code.groups]))
                self.specs.Samples.data += [((tr, cv), (cv, tr))]
            elif code.method == 'GroupKFold':
                self.specs.Samples.data.append(tuple(GroupKFold(**code.kwargs)
                                                     .split(self.X, self.Y, groups=self.Y[code.groups])))
            else:
                assert False, f'sampling method "{code.method}" not implemented'
        
    def run(self):
        for feats, (samps, sampsEnc) in product(self.specs.Features.data,
                                                zip(self.specs.Samples.data, self.specs.Samples.enc)):
            pm, i = self.mm.PM(feats, samps, get_i=True) # this also saves feats and samps
            self.mm.S.save(Samps=samps, Ctor=sampsEnc) #TODO maybe refactor this saving Ctor part out to somewhere better?
            
            self.setup_training(feats, samps, i=i)
            search = ParamSearch(self.specs.search)
            loop = search.search()
            for params in loop:
                paramsUse = dict(**self.specs.Params.data, **params)
                self.train(paramsUse) # sets some state attributes in self: self._save
                pm.save(Params=params, **self._save)
                loop.send(self._save.Results['score'])
                #del self._save #TODO probably wanna uncomment this in production
                
    def setup_training(self, feats, samps, *, i=None):
        samps = self.mm.S.get('Samps', i=i[1]) # this `samps` is a tuple of tuple of frozensets
        _X = self.X[feats]
        _dummy = pd.Series(range(len(_X)), index=_X.index)
        _s = O()
        _s.tr, _s.cv = tuple(_dummy.isin(s[0]) for s in samps), tuple(_dummy.isin(s[1]) for s in samps)
        lgb_data_info = dict(
            feature_name = list(_X.columns),
            categorical_feature = list(_X.dtypes[_X.dtypes.isin([np.int64,np.int32])].index),
            free_raw_data = False,
        )
        _L = self._L = O()
        _L.tr = [lgb.Dataset(_X[tr], P[self.specs.model.target][tr], **lgb_data_info,
                            **({'weight': P[self.specs.model.weight][tr]} if 'weight' in self.specs.model else {}))
                for tr in _s.tr]
        _L.cv = [lgb.Dataset(_X[cv], P[self.specs.model.target][cv], reference=Ltr, **lgb_data_info,
                            **({'weight': P[self.specs.model.weight][cv]} if 'weight' in self.specs.model else {}))
                for cv, Ltr in zip(_s.cv, _L.tr)]
        m = self.specs.metric
        if hasattr(m, 'attach'):
            m.attach(self)
        #TODO implement both logloss and Kaggle metric, and stop only when both don't improve in whatever num rounds
                
    def train(self, params):
        def iter_samples():
            for Ltr, Lcv in zip(self._L.tr, self._L.cv):
                evals_result = {}
                bst = lgb.train(params, Ltr, valid_sets=[Ltr, Lcv], valid_names=['tr', 'cv'],
                          feval=self.specs.metric, evals_result=evals_result, verbose_eval=1)
                df_results = (pd.DataFrame(evals_result['tr']), pd.DataFrame(evals_result['cv']))
                yield bst, df_results
        bsts, dfs = zip(*iter_samples())
        
        class save(O()):
            Training = dfs
            Boosters = bsts
            Results = dict(score=1)
        self._save = save

### specs for Search logic

In [325]:
class LPS(O()):
    class Discrete(O()):
        enc = {
            'learning_rate': [.05],
            ('max_depth','num_leaves'): #10
                [(6,1<<6),(9,1<<7),(9,1<<9),(12,1<<8),(12,1<<10),(12,1<<12),(-1,1<<8),(-1,1<<10),(-1,1<<12),(-1,1<<14)],
            #('max','num'): #10
            #    [(6,6),(9,7),(9,9),(12,8),(12,10),(12,12),(-1,8),(-1,10),(-1,12),(-1,14)],
        }
        
    class OneByOne(O()):
        class info(O()):
            a = "main [a]rray data of 3 values, [min mid max]"
            b =  "[b]ack up value, i.e. default value if array doesn't give better results"
            cast = "function to apply to values before using to cast to the right dtype"
            lim = "maximum number of iterations of searching in this hyperparameter"
        class default(O()):
            cast = keepSigFig(2)
            lim = 2
        data = {
            'min_data_in_leaf': O(a=[1,60,375], cast=round),
            'min_sum_hessian_in_leaf': O(a=[0,50,200]),
            'lambda_l1': O(a=[0,.02,.2], b=0, lim=1),
            'lambda_l2': O(a=[0,.02,.2], b=0, lim=1),
        }


class LMS(O()):
    model = O(
        time = 'time',
        value = 'y',
        target = 'target',
        weight = 'weight1',
    )
    
    metric = KaggleMetric()
    search = LPS
    
    class Features(O()):
        '''features selection groups'''
        data = [
            ['f1','f2','f3'],
            ['f4','f5','f6'],
        ]
    
    class Samples(O()):
        '''sample learning/cv split'''
        enc = [
            O(method='GroupShuffleSplit.2', kwargs=dict(n_splits=1, test_size=.5, random_state=44), groups='quarter'),
        ]
        
    class Params(O()):
        '''parameters constant settings'''
        data = dict(
            objective = 'binary',
            num_iterations = 100_000,
            early_stopping_round = 50,
            metric = None,
            seed = 44,
            bagging_seed = 45,
            feature_fraction_seed = 46,
        )

# Live Run Testing

### making test variables

In [326]:
from scipy.stats import norm
ppf = norm.ppf
eps = np.finfo(float).eps

In [327]:
def make_new_vars():
    n = 1000
    ##################RANDOMIZE#######################
    while True:
        try:
            u = np.random.uniform(size=(5, n))
            assert (u!=0).all()
            break
        except AssertionError:
            continue
    uu = u.copy()
    uu[0][np.random.randint(252, size=n) == 0] = np.nan
    uu[1][np.random.randint(173, size=n) == 0] = np.nan
    uu[2][np.random.randint(81, size=n) == 0] = np.nan
    uu[3][np.random.randint(27, size=n) == 0] = np.nan
    #################ENDRANDOM######################
    F = pd.DataFrame({'hax': np.arange(n)})
    F['time'] = F.hax // 10
    F['assetCodeId'] = F.hax % 10
    F['quarter'] = F.hax // 100
    P = F.copy()
    F['alex'] = ppf(u[0]*5%.99+eps)*ppf(u[1]*7%.99+eps)
    F['bob'] = ppf(.5+.49*uu[2]*np.sin(F.assetCodeId.values**10))*np.exp(uu[3])
    F['carol'] = -np.log(u[0]) + 4 * np.sin(1/uu[1])
    F['dean'] = u[1] + 2 * uu[2] - np.exp(-uu[3])
    F['edgar'] = ppf( (uu[2]**2-2*u[1]+(u[0]-u[3])**3-.5*u[1]**2) % .99 + eps )
    P['y'] = (ppf((u[0]+u[1])/2) + ppf((u[2]+u[3])/2) + .2*ppf(u[4])) * 1e-1
    P['universe'] = (~np.isnan(uu[0])).astype(float)
    return F, P

In [328]:
#F, P = make_new_vars()
#Path('lighttest.pkl').write_bytes(pickle.dumps((F, P)))
F, P = pickle.loads(Path('lighttest.pkl').read_bytes())

In [329]:
P['target'] = P.y>0
P['upDown'] = (P.target*2-1)
P['upDown1'] = P.upDown*P.universe.astype(int)
P['absVal'] = np.abs(P.y)
P['absVal1'] = P.absVal*P.universe
P['weight'] = P.absVal#.qtl()
P['weight1'] = P.weight*P.universe

In [330]:
F.head()

,hax,time,assetCodeId,quarter,alex,bob,carol,dean,edgar
0,0,0,0,0,-0.012198,0.000000,3.655924,1.336200,0.601672
1,1,0,1,0,-0.033782,1.139956,8.008799,1.244428,0.349115
2,2,0,2,0,0.034335,-0.333275,2.013295,1.549419,1.363992
3,3,0,3,0,-2.131567,-0.049046,5.325198,0.450859,0.940187
4,4,0,4,0,1.636082,0.267445,4.698875,0.670703,-0.449314


In [331]:
P.head()

,hax,time,assetCodeId,quarter,y,universe,target,upDown,upDown1,absVal,absVal1,weight,weight1
0,0,0,0,0,0.076738,1.0,True,1,1,0.076738,0.076738,0.076738,0.076738
1,1,0,1,0,0.048210,1.0,True,1,1,0.048210,0.048210,0.048210,0.048210
2,2,0,2,0,0.089397,1.0,True,1,1,0.089397,0.089397,0.089397,0.089397
3,3,0,3,0,-0.028319,1.0,False,-1,-1,0.028319,0.028319,0.028319,0.028319
4,4,0,4,0,-0.103184,1.0,False,-1,-1,0.103184,0.103184,0.103184,0.103184


In [332]:
class LPS(O()):
    class Discrete(O()):
        enc = {
            'learning_rate': [.05],
            ('max_depth','num_leaves'):
                [(3,1<<3),(6,1<<6),(-1,1<<9)]
        }
        
    class OneByOne(O()):
        class info(O()):
            a = "main [a]rray data of 3 values, [min mid max]"
            b =  "[b]ack up value, i.e. default value if array doesn't give better results"
            cast = "function to apply to values before using to cast to the right dtype"
            lim = "maximum number of iterations of searching in this hyperparameter"
        class default(O()):
            cast = keepSigFig(2)
            lim = 2
        data = {
            'min_data_in_leaf': O(a=[1,6,37], cast=round),
            'min_sum_hessian_in_leaf': O(a=[0,5,20]),
            'lambda_l1': O(a=[0,.02,.2], b=0, lim=1),
            'lambda_l2': O(a=[0,.02,.2], b=0, lim=1),
        }


class LMS(O()):
    model = O(
        time = 'time',
        value = 'y',
        target = 'target',
        weight = 'weight1',
    )
    
    metric = KaggleMetric()
    search = LPS
    
    class Features(O()):
        '''features selection groups'''
        data = [
            ['alex', 'bob', 'carol'],
            ['bob', 'dean', 'edgar'],
            ['alex', 'carol', 'edgar']
        ]
    
    class Samples(O()):
        '''sample learning/cv split'''
        enc = [
            O(method='GroupShuffleSplit.2', kwargs=dict(n_splits=1, test_size=.5, random_state=44), groups='quarter'),
        ]
        
    class Params(O()):
        '''parameters constant settings'''
        data = dict(
            objective = 'binary',
            num_iterations = 100_000,
            early_stopping_round = 50,
            metric = None,
            seed = 44,
            bagging_seed = 45,
            feature_fraction_seed = 46,
        )

In [334]:
mm = ModelManager('/big/data/search/test')
ms = ModelSearch(specs=LMS, mm=mm, X=F, Y=P)
ms.run()

[1]	tr's binary_logloss: 0.691463	cv's binary_logloss: 0.691594
Training until validation scores don't improve for 50 rounds.
[2]	tr's binary_logloss: 0.69314	cv's binary_logloss: 0.693401
[3]	tr's binary_logloss: 0.698143	cv's binary_logloss: 0.698535
[4]	tr's binary_logloss: 0.706417	cv's binary_logloss: 0.70694
[5]	tr's binary_logloss: 0.717888	cv's binary_logloss: 0.718542
[6]	tr's binary_logloss: 0.732462	cv's binary_logloss: 0.733247
[7]	tr's binary_logloss: 0.750029	cv's binary_logloss: 0.750945
[8]	tr's binary_logloss: 0.770468	cv's binary_logloss: 0.771515
[9]	tr's binary_logloss: 0.793645	cv's binary_logloss: 0.794822
[10]	tr's binary_logloss: 0.819418	cv's binary_logloss: 0.820726
[11]	tr's binary_logloss: 0.84764	cv's binary_logloss: 0.849079
[12]	tr's binary_logloss: 0.878163	cv's binary_logloss: 0.879732
[13]	tr's binary_logloss: 0.910834	cv's binary_logloss: 0.912535
[14]	tr's binary_logloss: 0.945506	cv's binary_logloss: 0.947337
[15]	tr's binary_logloss: 0.982031	cv's 

[17]	tr's binary_logloss: 0.527359	cv's binary_logloss: 0.574417
[18]	tr's binary_logloss: 0.522285	cv's binary_logloss: 0.572331
[19]	tr's binary_logloss: 0.517825	cv's binary_logloss: 0.570085
[20]	tr's binary_logloss: 0.513441	cv's binary_logloss: 0.566059
[21]	tr's binary_logloss: 0.509399	cv's binary_logloss: 0.564585
[22]	tr's binary_logloss: 0.505908	cv's binary_logloss: 0.562741
[23]	tr's binary_logloss: 0.502149	cv's binary_logloss: 0.559115
[24]	tr's binary_logloss: 0.498753	cv's binary_logloss: 0.558033
[25]	tr's binary_logloss: 0.495593	cv's binary_logloss: 0.554942
[26]	tr's binary_logloss: 0.49275	cv's binary_logloss: 0.553586
[27]	tr's binary_logloss: 0.489928	cv's binary_logloss: 0.552639
[28]	tr's binary_logloss: 0.487506	cv's binary_logloss: 0.551576
[29]	tr's binary_logloss: 0.484362	cv's binary_logloss: 0.549023
[30]	tr's binary_logloss: 0.481783	cv's binary_logloss: 0.546237
[31]	tr's binary_logloss: 0.479471	cv's binary_logloss: 0.545715
[32]	tr's binary_logloss: 

[81]	tr's binary_logloss: 0.434229	cv's binary_logloss: 0.493293
[82]	tr's binary_logloss: 0.43376	cv's binary_logloss: 0.493312
[83]	tr's binary_logloss: 0.433133	cv's binary_logloss: 0.492784
[84]	tr's binary_logloss: 0.432596	cv's binary_logloss: 0.492644
[85]	tr's binary_logloss: 0.43215	cv's binary_logloss: 0.492683
[86]	tr's binary_logloss: 0.43159	cv's binary_logloss: 0.492208
[87]	tr's binary_logloss: 0.431133	cv's binary_logloss: 0.492126
[88]	tr's binary_logloss: 0.43073	cv's binary_logloss: 0.492172
[89]	tr's binary_logloss: 0.430235	cv's binary_logloss: 0.492043
[90]	tr's binary_logloss: 0.429728	cv's binary_logloss: 0.49161
[91]	tr's binary_logloss: 0.429342	cv's binary_logloss: 0.491713
[92]	tr's binary_logloss: 0.428881	cv's binary_logloss: 0.491385
[93]	tr's binary_logloss: 0.42857	cv's binary_logloss: 0.49132
[94]	tr's binary_logloss: 0.428136	cv's binary_logloss: 0.491205
[95]	tr's binary_logloss: 0.427791	cv's binary_logloss: 0.491321
[96]	tr's binary_logloss: 0.4273

[76]	tr's binary_logloss: 4.00616	cv's binary_logloss: 3.99661
[77]	tr's binary_logloss: 4.05885	cv's binary_logloss: 4.04917
[78]	tr's binary_logloss: 4.11154	cv's binary_logloss: 4.10173
[79]	tr's binary_logloss: 4.16423	cv's binary_logloss: 4.1543
[80]	tr's binary_logloss: 4.21693	cv's binary_logloss: 4.20687
[81]	tr's binary_logloss: 4.26962	cv's binary_logloss: 4.25944
[82]	tr's binary_logloss: 4.32232	cv's binary_logloss: 4.31201
[83]	tr's binary_logloss: 4.37502	cv's binary_logloss: 4.36458
[84]	tr's binary_logloss: 4.42772	cv's binary_logloss: 4.41716
[85]	tr's binary_logloss: 4.48042	cv's binary_logloss: 4.46973
[86]	tr's binary_logloss: 4.53312	cv's binary_logloss: 4.52231
[87]	tr's binary_logloss: 4.58582	cv's binary_logloss: 4.57489
[88]	tr's binary_logloss: 4.63853	cv's binary_logloss: 4.62746
[89]	tr's binary_logloss: 4.69123	cv's binary_logloss: 4.68004
[90]	tr's binary_logloss: 4.74394	cv's binary_logloss: 4.73262
[91]	tr's binary_logloss: 4.79664	cv's binary_logloss: 4

[6]	tr's binary_logloss: 0.732462	cv's binary_logloss: 0.733247
[7]	tr's binary_logloss: 0.750029	cv's binary_logloss: 0.750945
[8]	tr's binary_logloss: 0.770468	cv's binary_logloss: 0.771515
[9]	tr's binary_logloss: 0.793645	cv's binary_logloss: 0.794822
[10]	tr's binary_logloss: 0.819418	cv's binary_logloss: 0.820726
[11]	tr's binary_logloss: 0.84764	cv's binary_logloss: 0.849079
[12]	tr's binary_logloss: 0.878163	cv's binary_logloss: 0.879732
[13]	tr's binary_logloss: 0.910834	cv's binary_logloss: 0.912535
[14]	tr's binary_logloss: 0.945506	cv's binary_logloss: 0.947337
[15]	tr's binary_logloss: 0.982031	cv's binary_logloss: 0.983993
[16]	tr's binary_logloss: 1.02027	cv's binary_logloss: 1.02236
[17]	tr's binary_logloss: 1.06008	cv's binary_logloss: 1.0623
[18]	tr's binary_logloss: 1.10134	cv's binary_logloss: 1.10369
[19]	tr's binary_logloss: 1.14391	cv's binary_logloss: 1.1464
[20]	tr's binary_logloss: 1.1877	cv's binary_logloss: 1.19031
[21]	tr's binary_logloss: 1.23258	cv's bina

...

[67]	tr's binary_logloss: 3.53213	cv's binary_logloss: 3.52371
[68]	tr's binary_logloss: 3.58478	cv's binary_logloss: 3.57623
[69]	tr's binary_logloss: 3.63744	cv's binary_logloss: 3.62876
[70]	tr's binary_logloss: 3.6901	cv's binary_logloss: 3.6813
[71]	tr's binary_logloss: 3.74277	cv's binary_logloss: 3.73384
[72]	tr's binary_logloss: 3.79544	cv's binary_logloss: 3.78639
[73]	tr's binary_logloss: 3.84811	cv's binary_logloss: 3.83894
[74]	tr's binary_logloss: 3.90079	cv's binary_logloss: 3.89149
[75]	tr's binary_logloss: 3.95347	cv's binary_logloss: 3.94405
[76]	tr's binary_logloss: 4.00616	cv's binary_logloss: 3.99661
[77]	tr's binary_logloss: 4.05885	cv's binary_logloss: 4.04917
[78]	tr's binary_logloss: 4.11154	cv's binary_logloss: 4.10173
[79]	tr's binary_logloss: 4.16423	cv's binary_logloss: 4.1543
[80]	tr's binary_logloss: 4.21693	cv's binary_logloss: 4.20687
[81]	tr's binary_logloss: 4.26962	cv's binary_logloss: 4.25944
[82]	tr's binary_logloss: 4.32232	cv's binary_logloss: 4.3

[43]	tr's binary_logloss: 2.27472	cv's binary_logloss: 2.26931
[44]	tr's binary_logloss: 2.32656	cv's binary_logloss: 2.32103
[45]	tr's binary_logloss: 2.3785	cv's binary_logloss: 2.37285
[46]	tr's binary_logloss: 2.43052	cv's binary_logloss: 2.42474
[47]	tr's binary_logloss: 2.48261	cv's binary_logloss: 2.4767
[48]	tr's binary_logloss: 2.53477	cv's binary_logloss: 2.52874
[49]	tr's binary_logloss: 2.58699	cv's binary_logloss: 2.58083
[50]	tr's binary_logloss: 2.63925	cv's binary_logloss: 2.63297
[51]	tr's binary_logloss: 2.69157	cv's binary_logloss: 2.68515
[52]	tr's binary_logloss: 2.74392	cv's binary_logloss: 2.73738
[53]	tr's binary_logloss: 2.79631	cv's binary_logloss: 2.78965
[54]	tr's binary_logloss: 2.84874	cv's binary_logloss: 2.84195
[55]	tr's binary_logloss: 2.9012	cv's binary_logloss: 2.89428
[56]	tr's binary_logloss: 2.95368	cv's binary_logloss: 2.94664
[57]	tr's binary_logloss: 3.00619	cv's binary_logloss: 2.99902
[58]	tr's binary_logloss: 3.05871	cv's binary_logloss: 3.0

[28]	tr's binary_logloss: 1.51883	cv's binary_logloss: 1.51531
[29]	tr's binary_logloss: 1.56709	cv's binary_logloss: 1.56344
[30]	tr's binary_logloss: 1.6158	cv's binary_logloss: 1.61203
[31]	tr's binary_logloss: 1.66492	cv's binary_logloss: 1.66103
[32]	tr's binary_logloss: 1.71441	cv's binary_logloss: 1.71039
[33]	tr's binary_logloss: 1.76423	cv's binary_logloss: 1.76008
[34]	tr's binary_logloss: 1.81435	cv's binary_logloss: 1.81008
[35]	tr's binary_logloss: 1.86474	cv's binary_logloss: 1.86034
[36]	tr's binary_logloss: 1.91536	cv's binary_logloss: 1.91084
[37]	tr's binary_logloss: 1.96621	cv's binary_logloss: 1.96156
[38]	tr's binary_logloss: 2.01725	cv's binary_logloss: 2.01247
[39]	tr's binary_logloss: 2.06846	cv's binary_logloss: 2.06355
[40]	tr's binary_logloss: 2.11983	cv's binary_logloss: 2.1148
[41]	tr's binary_logloss: 2.17134	cv's binary_logloss: 2.16618
[42]	tr's binary_logloss: 2.22297	cv's binary_logloss: 2.21769
[43]	tr's binary_logloss: 2.27472	cv's binary_logloss: 2.

[224]	tr's binary_logloss: 0.557831	cv's binary_logloss: 0.577081
[225]	tr's binary_logloss: 0.557757	cv's binary_logloss: 0.577183
[226]	tr's binary_logloss: 0.557682	cv's binary_logloss: 0.577117
[227]	tr's binary_logloss: 0.557607	cv's binary_logloss: 0.577011
[228]	tr's binary_logloss: 0.557533	cv's binary_logloss: 0.576947
[229]	tr's binary_logloss: 0.55746	cv's binary_logloss: 0.576843
[230]	tr's binary_logloss: 0.557388	cv's binary_logloss: 0.576781
[231]	tr's binary_logloss: 0.557317	cv's binary_logloss: 0.576679
[232]	tr's binary_logloss: 0.557246	cv's binary_logloss: 0.576868
[233]	tr's binary_logloss: 0.557176	cv's binary_logloss: 0.576809
[234]	tr's binary_logloss: 0.557106	cv's binary_logloss: 0.576708
[235]	tr's binary_logloss: 0.557037	cv's binary_logloss: 0.576737
[236]	tr's binary_logloss: 0.556968	cv's binary_logloss: 0.576679
[237]	tr's binary_logloss: 0.5569	cv's binary_logloss: 0.576581
[238]	tr's binary_logloss: 0.556833	cv's binary_logloss: 0.576524
[239]	tr's bi

[34]	tr's binary_logloss: 1.81435	cv's binary_logloss: 1.81008
[35]	tr's binary_logloss: 1.86474	cv's binary_logloss: 1.86034
[36]	tr's binary_logloss: 1.91536	cv's binary_logloss: 1.91084
[37]	tr's binary_logloss: 1.96621	cv's binary_logloss: 1.96156
[38]	tr's binary_logloss: 2.01725	cv's binary_logloss: 2.01247
[39]	tr's binary_logloss: 2.06846	cv's binary_logloss: 2.06355
[40]	tr's binary_logloss: 2.11983	cv's binary_logloss: 2.1148
[41]	tr's binary_logloss: 2.17134	cv's binary_logloss: 2.16618
[42]	tr's binary_logloss: 2.22297	cv's binary_logloss: 2.21769
[43]	tr's binary_logloss: 2.27472	cv's binary_logloss: 2.26931
[44]	tr's binary_logloss: 2.32656	cv's binary_logloss: 2.32103
[45]	tr's binary_logloss: 2.3785	cv's binary_logloss: 2.37285
[46]	tr's binary_logloss: 2.43052	cv's binary_logloss: 2.42474
[47]	tr's binary_logloss: 2.48261	cv's binary_logloss: 2.4767
[48]	tr's binary_logloss: 2.53477	cv's binary_logloss: 2.52874
[49]	tr's binary_logloss: 2.58699	cv's binary_logloss: 2.5

# -.-.-.-.-.-.-.-.-.-.-.-.-.- old shit / testing -.-.-.-.-.-.-.-.-.-.-.-.-.-.-

In [105]:
# PUREL TESTING
class HI(O()):
    class DERP():
        val = -99
        dir = Path('.')
        def yo(x):
            def y(a):
                return x(a) + 1000
            return y
        my = O(what=yo(lambda a: a ** 2))
        hm = O(func=lambda x: x * __class__.val, goto=lambda x: __class__.dir/x)
        class WHAT(O()):
            dude = lambda x: x + 10
        class YES(O()):
            dude = WHAT.dude
HI.DERP.my.what(5)
HI.DERP.hm.func(5)
HI.DERP.dir = Path('what')
HI.DERP.hm.goto('.git')
HI.DERP.WHAT.dude(0)

# ANOTHER PURE TESTING
def dothe(self):
    self.val = 4
    top = 100
    class Derp(O()):
        def func(x):
            return x + self.val
        def top(y):
            return y * top
    self.derp = Derp
item = Stop()
dothe(item)
item.derp.top(3)

class Yo(O()):
    class What(O()):
        derp = 5
    class Another(O()):
        yep = What.derp
Yo()

NameError: name 'What' is not defined